In [ ]:
import yaml
import boto3
import h5py
import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# general structure for scenes in s3
# s3_bucket/software/dem/scene/scene_files
s3_bucket = 'deant-data-public-dev'
aws_credentials = 'credentials/credentials_aws.yaml'

with open(aws_credentials, "r", encoding='utf8') as f:
    aws_cfg = yaml.safe_load(f.read())

# setup s3
s3 = boto3.client(
    's3', 
    region_name=aws_cfg['AWS_DEFAULT_REGION'],
    aws_access_key_id=aws_cfg['AWS_ACCESS_KEY_ID'],
    aws_secret_access_key= aws_cfg['AWS_SECRET_ACCESS_KEY'])

# make data directory to store local files
data_dir = '/data/scratch'
os.makedirs(data_dir, exist_ok=True)

In [ ]:
# set the scene we want to look at
software = 'rtc-opera'
dem = 'glo_30'
crs = '3031'
scene = 'S1B_IW_SLC__1SSH_20190315T195015_20190315T195045_015369_01CC73_DB8B'
h5_file = f'OPERA_L2_RTC-{scene}.h5'
s3_prefix = f'{software}/{dem}/{crs}/{scene}/{h5_file}'
local_path = f'{data_dir}/{h5_file}'
# download file
download = True
if download:
    print(f'downloading {s3_prefix}')
    s3.download_file(s3_bucket, s3_prefix, local_path)


In [ ]:
# load the h5 file
h5_data = h5py.File(local_path,'r')
print('keys')
print(list(h5_data))
print('data keys')
print(list(h5_data['data'].keys()))
print('metadata keys')
print(list(h5_data['metadata'].keys()))
print(list(h5_data['metadata']['processingInformation']['parameters'].keys()))

In [ ]:
def plot_array(
    arr, 
    cmap='binary_r',
    vmin=None,
    vmax=None,
    title='',
    ylabel='',
    save_path=''):

    stats_arr = np.array(arr)[np.array((np.isfinite(arr)))]
    print('Array Stats (excl nodata)')
    print(f'min: {stats_arr.min()}', 
        f'max: {stats_arr.max()}',
        f'mean: {stats_arr.mean()}',
        f'median: {np.percentile(stats_arr, 50)}',
        f'5th percentile: {np.percentile(stats_arr, 5)}',
        f'90th percentile: {np.percentile(stats_arr, 95)}',
        )

    # calculate percentiles if vmin or vmax is set as 'PXX'
    # e.g. vmin = 'P5'
    if vmin:
        if str(vmin[0]).upper() == 'P':
            vmin = np.percentile(stats_arr,int(vmin[1:]))
    if vmax:
        if str(vmax[0]).upper() == 'P':
            vmax = np.percentile(stats_arr,int(vmax[1:]))

    f, ax = plt.subplots(figsize=(10,10))
    im = ax.imshow(arr, 
            vmin = vmin, 
            vmax = vmax,
            cmap = cmap)
    ax.set_title(title)
    f.colorbar(im, ax=ax, label=ylabel)
    if save_path:
        plt.savefig(save_path)

In [ ]:
for i,k in enumerate(h5_data['data'].keys()):
    print(k)
    try:
        plot_array(h5_data['data'][k],
                save_path=os.path.join(data_dir,f'{scene}_{k}.jpg'),
                vmin='P5',
                vmax='P95')
    except:
        f'skipping, no image for: {k}'
    # if i == 0:
    #     break